#텐서(TENSOR)


---

<br>

배열이나 행렬과 매우 유사한 특수한 자료구조.

**PyTorch**에서는 텐서를 사용하여 모델의 입출력, 그리고 모델의 매개변수들을 부호화(encode)한다.

Tensor는 GPU나 다른 하드웨어 가속기에서 실행할 수 있다는 점을 제외하면, NumPy의 ndarray와 유사함.

실제로, Tensor랑 ndarray가 종종(?) 동**일한 내부(underly)메모리를 공유할 수 있어서**, data를 복사할 필요가 없다. tensor의 값을 바꾸면, 그 tensor를 numpy로 바꾼 ndarray의 값도 **자동으로 업데이트** 됨.

또, tensor는 **자동미분(automatic differentiation)에 최적화** 되어 있음.

In [ ]:
import torch
import numpy as np

### **- 텐서 초기화**

여러가지 방법으로 tensor를 초기화 할 수 있음.

**1. 데이터로부터 직접(directly) 생성**

In [ ]:
# 일단 집어넣으면 자료형은 알아서 유추함
data = [[1,2], [3,4]] #아 그럼 tensor는 list네..?
                      #그렇다 tensor는 3차원 리스트이다
x_data = torch.tensor(data)

**2. NumPy 배열로부터 생성**

tensor ↔ numpy 변환 가능.

In [ ]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

**3. 다른 tensor로부터 생성.**

메소드 이름 뒤에 '_like'가 붙는 거 사용.(ones_like()라던가)

명시적으로 재정의(override)하지 않는다면, 인자로 주어진 텐서의 속성(shape, datatype)을 유지

In [ ]:
#x_data shape: [2, 2], dtype: LongTensor(64비트 int)
x_ones = torch.ones_like(x_data)
print(f"Ones Tensor: \n {x_ones} \n") #torch.ones => 1값을 갖는 tensor 생성.
                                      #torch.ones(5) = tensor([1.,1.,1.,1.,1.,])

x_rand = torch.rand_like(x_data, dtype=torch.float)# x_data 속성 덮어쓰기
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.0641, 0.8724],
        [0.8609, 0.6859]]) 



**4. 무작위(random) 또는 상수(constant) 값을 사용.**


아래 코드의 shape => 텐서의 dimension을 나타내는 튜플.

In [ ]:
shape=(2,3,) #shape값을 튜플로 미리 지정한 후, tensor 생성시 사용

rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.6297, 0.2243, 0.3904],
        [0.0432, 0.1353, 0.1022]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


### **- 텐서의 속성(Attribute)**

텐서 속성 종류: **모양(shape), 자료형(datatype)**

**모양**을 알고 싶을 때는 **.shape**
**자료형**을 알고 싶을 때는 **.dtype**

In [ ]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}\n")
print(f"Datatype of tensor: {tensor.dtype}\n")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])

Datatype of tensor: torch.float32

Device tensor is stored on: cpu


### **- 텐서 연산(Operation)**

전치(transposing), 인덱싱(indexing) 등 100가지 이상의 텐서 연산을 할 수 있음.

각 연산들은 (일반적으로 CPU보다 빠른)**GPU에서 실행할 수 있음**. 코랩에서는 Edit>Notebook>Settings가면 GPU 할당할 수 있음.

기본적으로 tensor는 CPU에 생성되는데, .to메소드 사용하면 (GPU의 가용성(availabilty) 확인한 후) GPU로 텐서를 명시적으로 이동시킬 수 있음.

**주의 - 큰 텐서들을 장치들 간에 복사하면 시간, 메모리 측면에서 비용이 많이 듦**

In [ ]:
if torch.cuda.is_available():
    tensor = tensor.to("cuda")

### **- 여러 텐서 연산**

In [ ]:
#Numpy식 표준 인덱싱과 슬라이싱
#tensor = torch.tensor(tensor)
tensor = torch.ones(4,4)
print(f"First row: {tensor[0]}\n") #indexing
print(f"First column: {tensor[:, 0]}\n") #tensor의 모든 행의 [0]번째 원소
print(f"Last column: {tensor[..., -1]}") #Ellipsis(...) in NumPy
                                         # => 요소나 범위를 지정할 때 Ellipsis( )를 사용하여 중간 차원을 생략할 수 있음
                                         #tensor[n]의 중간 index들을 싸그리 흘려보낸 것
tensor[:,1] = 0

print(tensor)

First row: tensor([1., 1., 1., 1.])

First column: tensor([1., 1., 1., 1.])

Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [ ]:
#tensor 합치기
t1 = torch.cat([tensor, tensor, tensor], dim=1) #dim=0, 아래에 붙여준다. dim=1, 같은 행끼리 붙여준다.
print(t1)

'''
torch.cat()은 주어진 차원을 기준으로 주어진 텐서들을 붙입(concatenate)니다.
torch.stack()은 새로운 차원으로 주어진 텐서들을 붙입니다.
따라서, (3, 4)의 크기(shape)를 갖는 2개의 텐서 A와 B를 붙이는 경우,
torch.cat([A, B], dim=0)의 결과는 (6, 4)의 크기(shape)를 갖고, (대괄호를 추가하지 않음. 그대로 갖다 붙이기)
torch.stack([A, B], dim=0)의 결과는 (2, 3, 4)의 크기를 갖습니다.(각 A,B에 대괄호 두르고 붙이기)
'''

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


'\ntorch.cat()은 주어진 차원을 기준으로 주어진 텐서들을 붙입(concatenate)니다.\ntorch.stack()은 새로운 차원으로 주어진 텐서들을 붙입니다.\n따라서, (3, 4)의 크기(shape)를 갖는 2개의 텐서 A와 B를 붙이는 경우,\ntorch.cat([A, B], dim=0)의 결과는 (6, 4)의 크기(shape)를 갖고, (대괄호를 추가하지 않음. 그대로 갖다 붙이기)\ntorch.stack([A, B], dim=0)의 결과는 (2, 3, 4)의 크기를 갖습니다.(각 A,B에 대괄호 두르고 붙이기)\n'

In [ ]:
#산술 연산(Arithmetic operations)

# 두 텐서 간의 행렬 곱(matrix multiplication)을 계산합니다. y1, y2, y3은 모두 같은 값을 갖습니다.
# ``tensor.T`` 는 텐서의 전치(transpose)를 반환합니다.

y1 = tensor @ tensor.T #아래와 같은 뜻. @는 행렬곱을 뜻함.
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1) # 같은 shpae끼리 곱해서 그냥 rand_like로 만들고 거기다 값 넣은 듯
torch.matmul(tensor, tensor.T, out=y3)

print(y3)


# 요소별 곱(element-wise product)을 계산합니다. z1, z2, z3는 모두 같은 값을 갖습니다.

z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

print(z3)

#참고 - 내적은 .dot

tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [ ]:
#단일-요소(single-element) 텐서

#텐서의 모든 값을 하나로 집계(aggregate)해서 요소가 하나인 텐서의 경우, item()으로 숫자값 반환 가능.

agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

12.0 <class 'float'>


In [ ]:
#바꿔치기(in-place)연산

# +=, -=랑 똑같슴니다
#_접미사를 갖는다

print(f"{tensor} \n")
tensor.add_(5) #tensor = tensor * 5
print(tensor)

#주의 - 바꿔치기 연산은 메모리를 일부 절약하지만,
#       기록(history)이 즉시 삭제되어 도함수(derivative) 계산에 문제가 발생할 수 있습니다.
#       따라서, 사용을 권장하지 않습니다.

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])
